### **Importação das bibliotecas necessárias**

In [1]:
import os
import cv2
import random
import itertools
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, concatenate, add
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import log_loss, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

### **Leitura e pre-processamentos dos dados**

In [2]:
train_generator = ImageDataGenerator(rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

train_generator = train_generator.flow_from_directory(
        '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/train',  
        target_size=(150,150),  
        batch_size=64,
        class_mode='binary')

test_generator = ImageDataGenerator(rescale=1.0/255)

validation_generator = test_generator.flow_from_directory(
        '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/val',  
        target_size=(150,150),  
        batch_size=64,
        class_mode='binary')

test_generator = test_generator.flow_from_directory(
       '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/test',  
        target_size=(150,150),  
        batch_size=64,
        class_mode='binary')

## **Definindo funções de métricas de avaliação do modelo**

In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
def create_model():
    model_vgg16 = VGG16(weights='imagenet',
                        include_top=False, 
                        input_shape=(150, 150, 3))
    
    for layer in model_vgg16.layers[:]:
        layer.trainable = False

    for layer in model_vgg16.layers:
        print(layer, layer.trainable)
    
    model = Sequential()
    model.add(model_vgg16)
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))   
    model.summary()
    sgd = SGD(lr=1e-3)

    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=2e-5), metrics=['acc',f1_m])
    #model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['acc',f1_m])
    return model
    

In [9]:
model = create_model()

In [10]:
history = model.fit(
       train_generator,
       epochs=15,
       validation_data=test_generator)

In [11]:
acc = history.history['acc']
 
val_acc = history.history['val_acc']   
loss = history.history['loss']
val_loss = history.history['val_loss']   
epochs = range(1, len(acc) + 1) 
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy')
plt.legend(loc='best')

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc='best')


In [13]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [15]:
loss, acc, f1_m= model.evaluate(test_generator)

In [17]:
print('Accuracy:', acc)
print('Loss:', loss)
print('F1_score:', f1_m)